2)Problem Statement


It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?

Take this monstrosity as the DataFrame to use in the following puzzles:


df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

1)1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 'Budapest_PaRis', 'Brussels_londOn'], 'FlightNumber': [10045, np.nan, 10065, np.nan, 10085], 'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]], 'Airline': ['KLM(!)', ' (12)', '(British Airways. )', '12. Air France', '"Swiss Air"']})
df

Airline  FlightNumber           From_To  RecentDelays
0               KLM(!)       10045.0      LoNDon_paris      [23, 47]
1                 (12)           NaN      MAdrid_miLAN            []
2  (British Airways. )       10065.0  londON_StockhOlm  [24, 43, 87]
3       12. Air France           NaN    Budapest_PaRis          [13]
4          "Swiss Air"       10085.0   Brussels_londOn      [67, 32]

In [2]:
for i in range(1, df['FlightNumber'].count() + 1):
    if pd.isnull(df.loc[i,'FlightNumber']):
        df.loc[i,'FlightNumber'] = df.loc[i-1,'FlightNumber'] + 10

In [3]:
df

Airline  FlightNumber           From_To  RecentDelays
0               KLM(!)       10045.0      LoNDon_paris      [23, 47]
1                 (12)       10055.0      MAdrid_miLAN            []
2  (British Airways. )       10065.0  londON_StockhOlm  [24, 43, 87]
3       12. Air France       10075.0    Budapest_PaRis          [13]
4          "Swiss Air"       10085.0   Brussels_londOn      [67, 32]

In [4]:
df['FlightNumber'].astype(int)


0    10045
1    10055
2    10065
3    10075
4    10085
Name: FlightNumber, dtype: int32

2) The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame

In [5]:
df['From_To']

0        LoNDon_paris
1        MAdrid_miLAN
2    londON_StockhOlm
3      Budapest_PaRis
4     Brussels_londOn
Name: From_To, dtype: object

In [6]:
temporaryDF = df.copy()

In [7]:
temporaryDF[['FROM' ,'TO']] = temporaryDF.From_To.str.split('_',expand=True)
temporaryDF

Airline  FlightNumber           From_To  RecentDelays  \
0               KLM(!)       10045.0      LoNDon_paris      [23, 47]   
1                 (12)       10055.0      MAdrid_miLAN            []   
2  (British Airways. )       10065.0  londON_StockhOlm  [24, 43, 87]   
3       12. Air France       10075.0    Budapest_PaRis          [13]   
4          "Swiss Air"       10085.0   Brussels_londOn      [67, 32]   

       FROM         TO  
0    LoNDon      paris  
1    MAdrid      miLAN  
2    londON  StockhOlm  
3  Budapest      PaRis  
4  Brussels     londOn

3) Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

In [8]:
temporaryDF.From_To = temporaryDF.From_To.str.capitalize()
temporaryDF.FROM = temporaryDF.FROM.str.capitalize()
temporaryDF.TO = temporaryDF.TO.str.capitalize()
temporaryDF

Airline  FlightNumber           From_To  RecentDelays  \
0               KLM(!)       10045.0      London_paris      [23, 47]   
1                 (12)       10055.0      Madrid_milan            []   
2  (British Airways. )       10065.0  London_stockholm  [24, 43, 87]   
3       12. Air France       10075.0    Budapest_paris          [13]   
4          "Swiss Air"       10085.0   Brussels_london      [67, 32]   

       FROM         TO  
0    London      Paris  
1    Madrid      Milan  
2    London  Stockholm  
3  Budapest      Paris  
4  Brussels     London

4) Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [9]:
df.drop('From_To', axis=1,inplace=True)
df

Airline  FlightNumber  RecentDelays
0               KLM(!)       10045.0      [23, 47]
1                 (12)       10055.0            []
2  (British Airways. )       10065.0  [24, 43, 87]
3       12. Air France       10075.0          [13]
4          "Swiss Air"       10085.0      [67, 32]

In [10]:
df['From_To'] = temporaryDF['From_To']
df

Airline  FlightNumber  RecentDelays           From_To
0               KLM(!)       10045.0      [23, 47]      London_paris
1                 (12)       10055.0            []      Madrid_milan
2  (British Airways. )       10065.0  [24, 43, 87]  London_stockholm
3       12. Air France       10075.0          [13]    Budapest_paris
4          "Swiss Air"       10085.0      [67, 32]   Brussels_london

5) In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each 
second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [11]:
df2 = pd.DataFrame(df['RecentDelays'].tolist())
df2

0     1     2
0  23.0  47.0   NaN
1   NaN   NaN   NaN
2  24.0  43.0  87.0
3  13.0   NaN   NaN
4  67.0  32.0   NaN

In [12]:
df['delay_1'] = df2.loc[:,0]
df['delay_2'] = df2.loc[:,1]
df['delay_3'] = df2.loc[:,2]
df.drop(['RecentDelays'],axis=1, inplace=True)
df

Airline  FlightNumber           From_To  delay_1  delay_2  \
0               KLM(!)       10045.0      London_paris     23.0     47.0   
1                 (12)       10055.0      Madrid_milan      NaN      NaN   
2  (British Airways. )       10065.0  London_stockholm     24.0     43.0   
3       12. Air France       10075.0    Budapest_paris     13.0      NaN   
4          "Swiss Air"       10085.0   Brussels_london     67.0     32.0   

   delay_3  
0      NaN  
1      NaN  
2     87.0  
3      NaN  
4      NaN